<a href="https://colab.research.google.com/github/jayaemekar/CSCI-611-AppiledMachineLearning/blob/main/JELab3knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Name: Jayarani Emekar, CSCI-611 Date- 27 Feb 2021

 Sources/guidance/assistance received - Python tutorials scipy-lectures,Muller textbook

 Enviorment Developed and Tested: GOOGLE COLABORATORY (COLAB)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

TASK 1 - IMPLEMENTATION

Imports used in KNN class

In [ ]:
# imports used as per specifications
import numpy as np
import math
from decimal import Decimal 
from scipy.stats import mode 


Function to calculate euclidean distance

In [ ]:
# function to calculate euclidean distance
def euclidean(arr1, arr2):
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)
    return np.sqrt(np.sum((arr1-arr2)**2))

function to calculate manhattan distance

In [ ]:
# function to calculate manhattan distance    
def manhattan( arr1, arr2):
    arr1 = np.array(arr1)
    arr2 = np.array(arr2)
    return np.sum(np.abs(arr1-arr2))

function to calculate minkowski distance

In [ ]:
# function to calculate minkowski distance    
def p_root(value, root):    
    root_value = 1 / float(root) 
    return round (Decimal(value) **
             Decimal(root_value), 3) 
  
def minkowski(arr1, arr2, p_value):   
    return (p_root(sum(pow(abs(a-b), p_value) 
            for a, b in zip(arr1, arr2)), p_value)) 

Function to calculate quicksort distance

In [ ]:
# function to calculate quicksort distance   
def quicksort(lst, index):
    if lst.ndim == 1 and index != -1:
      return quicksort_1d(lst)
    elif lst.ndim == 1 and index == -1:
      return quicksort_indices(lst)
    else:
        return quicksort_2d(lst,index)

def quicksort_1d(lst):

    less = []
    equal = []
    greater = []

    if len(lst) > 1:
        pivot = lst[0]
        for x in lst:
            if x < pivot:
                less.append(x)
            elif x == pivot:
                equal.append(x)
            elif x > pivot:
                greater.append(x)
        return quicksort_1d(less)+equal+quicksort_1d(greater) 
    else:  
        return lst

def quicksort_indices(lst):
    return [i for (v, i) in sorted((v, i) for (i, v) in enumerate(lst))]

def quicksort_2d(lst, index):
    if len(lst) == 0:
        return []
    else:
        pivot = lst[0]
        lesser = quicksort_2d([x for x in lst[1:] if x[index] < pivot[index]], index)
        greater = quicksort_2d([x for x in lst[1:] if x[index] >= pivot[index]], index)
        return lesser + [pivot] + greater

knn class and knnBuild should create an object, with which to invoke the knnFit and knnPredict as methods

knnScore method - Bonus feature

weights---> Uniform and Distance - Bonus feature

In [ ]:
class KNN():

# initialize the KNN object
    def __init__(self, model, k, algorithm='euclidean',p=2,weights ='uniform'):
        self.model = model
        self.k = k
        self.algorithm = algorithm
        self.p = p
        self.weights = weights

# knnFit function
    def knnFit(self, X, y):
        self.X = X 
        self.y = y 
        # no_of_training_examples, no_of_features 
        self.m, self.n = X.shape
        return self

# This is the function to calculate the distance
    def _distance(self, data1, data2):
        """1: Manhattan, 2: Euclidean, 3: Minkowski"""
        if self.algorithm == 'euclidean':
            return euclidean(data1 , data2)          
        elif self.algorithm == 'manhattan':
            return manhattan(data1, data2)
        elif self.algorithm == 'minkowski':
            return minkowski(data1, data2 ,self.p)
        raise ValueError("algorithm not recognized")

# This is the function to compute the weights, distance/uniform (Bonus feature)
    def _compute_weights(self, distances):
        if self.weights == 'uniform':
            return [(1, y) for d, y in distances]
        elif self.weights == 'distance':
            matches = [(1, y) for d, y in distances if d == 0]
            return matches if matches else [(1/d, y) for d, y in distances]
        raise ValueError("weights not recognized: should be 'uniform' or 'distance'")

# This is knn predict function
    def knnPredict(self, X, y):
        if self.model == 'classifier':
            return self.predict_classifier(X)
        elif self.model == 'regressor':
            return self.predict_regressor(X)
        else:
            raise ValueError("Model not recognized: should be 'classifier' or 'regressor'")

# This is the function to calculate the knnScore (Bonus feature)
    def knnScore(self, X, y):
        if self.model == 'classifier':
            return sum(1 for p, t in zip(self.predict_classifier(X), y) if p == t) / len(y)
        elif self.model == 'regressor':
            return np.corrcoef(y, self.predict_regressor(X))[0,1]**2
        else:
            raise ValueError("Model not recognized: should be 'classifier' or 'regressor'")
    
# Function for regressor prediction 
    def predict_regressor( self, X_test ) : 
        self.X = X_test 
        # no_of_test_examples, no_of_features 
        self.m_test, self.n = X_test.shape 
        # initialize Y_predict 
        Y_predict = np.zeros( self.m_test ) 
        for i in range( self.m_test ) : 
            x = self.X[i] 
            # find the K nearest neighbors from current test example 
            neighbors = np.zeros( self.k ) 
            neighbors = self.find_neighbors_regressor( x ) 
            # calculate the mean of K nearest neighbors 
            Y_predict[i] = np.mean( neighbors ) 
        return Y_predict 

# Function for classifier prediction 
    def predict_classifier( self, X_test ) : 
        self.X_test = X_test 
        # no_of_test_examples, no_of_features 
        self.m_test, self.n = X_test.shape 
        # initialize Y_predict 
        Y_predict = np.zeros( self.m_test ) 
        for i in range( self.m_test ) : 
            x = self.X_test[i] 
            # find the K nearest neighbors from current test example 
            neighbors = np.zeros( self.k ) 
            neighbors = self.find_neighbors_classifier( x ) 
            # most frequent class in K neighbors 
            Y_predict[i] = mode( neighbors )[0][0]
        return Y_predict 

# Function to find the K nearest neighbors to current test example 
    def find_neighbors_regressor( self, x ) : 
        # calculate all the euclidean distances between current test example x and training set X_train 
        distances = np.zeros( self.m ) 
        for i in range( self.m ) : 
            d = self._distance(x, self.X[i])
            distances[i] = d 
        # sort Y_train according to distance_array and 
        inds = quicksort(distances, -1)
        # store into Y_train_sorted 
        Y_train_sorted = self.y[inds] 
        return Y_train_sorted[:self.k] 

# Function to find the K nearest neighbors to current test example 
    def find_neighbors_classifier( self, x ) : 
        # calculate all the euclidean distances between current test example x and training set X_train 
        distances = np.zeros( self.m ) 
        for i in range( self.m ) : 
            d = self._distance(x, self.X[i])
            distances[i] = d 
        # sort Y_train according to distance_array and  
        inds = quicksort(distances, -1)
        # store into Y_train_sorted
        Y_train_sorted = self.y[inds] 
        return Y_train_sorted[:self.k] 

class knnBuild(KNN):
    def __init__(self, model, k, algorithm='euclidean',p=2,weights ='uniform'):
	      KNN.__init__(self,model, k, algorithm='euclidean',p=2,weights ='uniform')